# Making An Air Pollutant Forecasting model - Part I

<br>
Before we make our own air pollutant forecasting model, let's first try to describe the relationship between individual air pollutants and population (or, predict the values of those counties without measurements). 
The data science process you learn here will be very useful for building a useful forecasting tool later!

We will start by:
- using yearly average AQI
- using only one "feature" (population) for prediction
- using simple linear models: a straight line and a quadratic function

Let's start by loading in some libaries using the __`import`__ function

In [1]:
import numpy as np
import pandas as pd
import importlib
from IPython.display import Image
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from bokeh.io import output_notebook
from bokeh.layouts import gridplot, row
from bokeh.plotting import figure, show
output_notebook()

Loading BokehJS ...

Next, we load in our data table, and create two additional attributes: Logarithm of population and its squared value. The table below should look identical to the one we see in CODAP.

In [2]:
df=pd.read_csv('popair_2017.csv')
df["Log_pop"]=pd.Series(np.log10(df["Population"]))
df["Log_pop_sq"]=pd.Series(df["Log_pop"]**2)
df

,Unnamed: 0,State_name,County_name,Year,avgAQI_o3,avgAQI_co,avgAQI_no2,avgAQI_so2,avgAQI_pm25,Population,Log_pop,Log_pop_sq
0,0,Alabama,Baldwin,2017,37.536170,NaN,NaN,NaN,30.218182,212628,5.327620,28.383540
1,1,Alabama,Clay,2017,NaN,NaN,NaN,NaN,32.457627,13367,4.126034,17.024156
2,2,Alabama,Colbert,2017,35.808943,NaN,NaN,NaN,30.487395,54500,4.736397,22.433452
3,3,Alabama,DeKalb,2017,36.844011,NaN,NaN,NaN,32.886957,71617,4.855016,23.571182
4,4,Alabama,Elmore,2017,34.013043,NaN,NaN,NaN,NaN,81677,4.912100,24.128724
...,...,...,...,...,...,...,...,...,...,...,...,...
917,917,Wyoming,Sublette,2017,45.682047,NaN,5.003844,NaN,22.925208,9799,3.991182,15.929532
918,918,Wyoming,Sweetwater,2017,41.713624,NaN,8.001370,3.638546,22.737705,43534,4.638829,21.518731
919,919,Wyoming,Teton,2017,41.922535,2.164384,NaN,NaN,15.535176,23265,4.366703,19.068096
920,920,Wyoming,Uinta,2017,44.126722,NaN,4.539726,NaN,NaN,20495,4.311648,18.590308


Here, we can select an attribute you want to predict.

Type in __avgAQI_o3__, __avgAQI_co__, __avgAQI_no2__, __avgAQI_so2__, or __avgAQI_pm25__ in the double quote below.

In [3]:
attr = "avgAQI_no2" 

In [4]:
#These couple lines are just selecting a subset of table with that attribute availalbe.
X=df.loc[:,["Log_pop","Log_pop_sq"]][df[attr]>0]
y=df.loc[:,attr][df[attr]>0]

We will split the data into a training set and a hold out set according to our analysis plan.
![analysisplan](analysis.png)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
print("Number of counties have measurements for the selected pollutant = ",len(y))
print("Number of counties in the training data set = ",len(y_train))
print("Number of counties in the hold out (testing) data set = ",len(y_test))

Number of counties have measurements for the selected pollutant =  242
Number of counties in the training data set =  193
Number of counties in the hold out (testing) data set =  49


### Model 1: A Straight Line $y=a+bx$

In [6]:
import lm1_interactive
importlib.reload(lm1_interactive)
lm1_interactive.lm1_interactive(X_train.iloc[:,0],y_train)

Loading BokehJS ...

Loading BokehJS ...

You can alculate the Mean-Squared-Error (__MSE__) based on the model. Try to run a few different combinations of a and b. Comparing the plots above and MSEs. What do you notice?

In [7]:
a = -24.26
b = 7.2

MSE = sum((a+b*X_train.iloc[:,0]-y_train)**2)/len(y_train)
print("MSE = "+np.str(np.around(MSE,2)))

MSE = 30.49


### Model 2: A Quadratic Function $y=a+bx+cx^2$

In [8]:
import lm2_interactive
importlib.reload(lm2_interactive)
lm2_interactive.lm2_interactive(X_train.iloc[:,0],y_train)

Loading BokehJS ...

Loading BokehJS ...

Again, you can alculate the Mean-Squared-Error (__MSE__) based on the model.

In [9]:
a = -3.72
b = -1.05
c = 0.81

MSE = sum((a+b*X_train.iloc[:,0]+c*X_train.iloc[:,0]**2-y_train)**2)/len(y_train)
print("MSE = "+np.str(np.around(MSE,2)))

MSE = 30.19


### Validate Model 1 and Model 2 using the hold out data set

We'll now validate the two models you explored earlier using the data set we intentionally set aside.

In [10]:
lr1 = LinearRegression().fit(np.array(X_train.iloc[:,0]).reshape(-1,1), y_train)
print("The best-fit line has an intercept (a) of "+np.str(np.around(lr1.intercept_,2))+
      " and a slope (b) of "+np.str(np.around(lr1.coef_[0],2)))

The best-fit line has an intercept (a) of -23.91 and a slope (b) of 7.07


In [11]:
lr2 = LinearRegression().fit(X_train, y_train)
print("The best-fit quadratic function has a, b, c "+ np.str(np.around(lr2.intercept_,2)), 
      np.str(np.around(lr2.coef_[0],2))+" and "+np.str(np.around(lr2.coef_[1],2)))

The best-fit quadratic function has a, b, c -1.89 -1.75 and 0.87


In [12]:
import lm_plot
importlib.reload(lm_plot)
lm_plot.lm_plot(X_train.iloc[:,0],y_train,X_test.iloc[:,0],y_test,lr1)
lm_plot.lm_plot(X_train.iloc[:,0],y_train,X_test.iloc[:,0],y_test,lr2)

Loading BokehJS ...

Loading BokehJS ...